# Programming Assignment #4: Checksums

In this assignment, we will explore various uses of modular arithmetic to construct checksums, which are used extensively in error-detection and error-correction codes.

As a general rule, there are two types of errors that can be detected or even better corrected. The first type is a change in one of the digits, e.g., a 7 can be changed to a 5. The sercond type is a swap of two adjacent digits, e.g., 147 can change to 417. Different techniques will detect and/or fix some of these errors.


## Airline Ticket Numbers

When you buy a plane ticket, you may find a ticket id (different from your confirmation number). This is a 15-digit number, where the 15th digit is a checksum. In particular, the 15th digit is chosen such that it is equal to the remainder of the 14-digit number (of the first 14 digits) modulo 7. For example, the following is a valid airline ticket number:

$$1 2 3 4 5 6 7 8 9 0 1 2 3 4 0$$

because $12345678901234 \bmod 7 = 0$.

### TODO-1 (10 points)

Write the function `(valid-ticket ID)` that verifies that the given ID, which is a list of 14 digits, is a valid airline ticket ID. For example,

    (valid-ticket '(1 2 3 4 5 6 7 8 9 0 1 2 3 4 0)) = T
    
> Hint: I found it useful to write a function that extracts the ticket number from a ticket. E.g., the ticket number of the ticket above is 12,345,678,901,234.

> Hint: You will need to define a data type for things like "digit" and "list of digits". Later, we will bd writing functions for things like "lists of natural" and try to pass a list of digits as an argument. Unfortunately, ACL2 will not allow this. Let's simplify things right now by telling ACL2 using `defthm` that a list of digits **is** a list of naturals. This will save you a lot of time later, so just do it now and forget it.

In [1]:
(defsnapshot from-the-top)
(defsnapshot todo-1)

(defdata digit
    (range integer (0 <= _ <= 9)))

(defdata list-digits
    (listof digit))

(definec listToNum (numbers :list-digits):nat
    (if (or (endp numbers) (equal (len numbers) 1))
        0
        (+ (* (expt 10 (- (len numbers) 2)) (first numbers)) (listToNum (rest numbers)))))

(definec valid-ticket (numbers :list-digits):bool
    (equal (mod (listToNum numbers) 7) (first(last numbers))))


(check-expect (listToNum '(1 2 3 4)) 123)
(check-expect (listToNum '(8 9 0 1)) 890)
(check-expect (listToNum '(1 2 3 4 5 6 7 8 9 0 1 2 3 4 0)) 12345678901234)
(check-expect (listToNum '(1 2 3)) 12)
(check-expect (listToNum '(1 2 3 4 7 3)) 12347)

(check-expect (valid-ticket '(1 2 3 4 5 6 7 8 9 0 1 2 3 4 0)) t)
(check-expect (valid-ticket '(1 2 2 2 2 2 2 6 7 8 9 6 3 4 9)) nil)
(check-expect (valid-ticket '(1 9 3 5 6 3 2 4 6 7 5 3 2 5 7)) nil)
(check-expect (valid-ticket '(8 3 9 5 4 8 9 0 3 2 5 4 8 9 1)) t)
(check-expect (valid-ticket '(4 3 8 9 6 5 7 3 0 2 1 5 6 9 2)) t)

ACL2S !>>(DEFSNAPSHOT FROM-THE-TOP)
          20:x(DEFMACRO DEFSNAPSHOT (LABEL) ...)

Summary
Form:  ( DEFLABEL FROM-THE-TOP ...)
Rules: NIL
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
 FROM-THE-TOP
ACL2S !>>(DEFSNAPSHOT TODO-1)

Summary
Form:  ( DEFLABEL TODO-1 ...)
Rules: NIL
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
 TODO-1
ACL2S !>>(DEFDATA DIGIT (RANGE INTEGER (0 <= _ <= 9)))
 Predicate events...
Form:  ( DEFUN DIGITP ...)
Form:  ( IN-THEORY (DISABLE* ...))
Form:  ( IN-THEORY (ENABLE ...))
Form:  ( TABLE ACL2::RULESET-TABLE ...)
Form:  ( MAKE-EVENT (LET* ...))
 Tau characterization events...
 (DIGITP ACL2::V1) <= body -- not complete. 
Reasons: ("Illegal tau rule") 
 (DIGITP ACL2::V1) => body -- not complete. 
Reasons: 
("The formula fails to fit any of the forms for acceptable :TAU-SYSTEM rules.")

Form:  ( DEFTHM DIGIT=>DEF ...)
 Enumerator events...
Form:  ( DEFUN NTH-DIGIT-BUILTIN ...)
Form:  ( DEFUN NTH-DIGIT/ACC-BUILTIN ...)
Form:  ( PROGN (

### TODO-2

We are interested in detecting errors, e.g., an invalid ticket number. What we want to show is that if a ticket number was written down incorrectly, e.g., with one of the common mistakes (changing a number or transposing two adjacent numbers), then the resulting ticket will be invalid.

To get started, we need to write functions that perform those two types of errors. I.e., write a function `(change-nth-digit-to ticket n d)` that returns a new ticket that is equal to the old ticket, except that the nth digit will be `d`. Also, write a function called `(transpose-nth-digit ticket n)` that returns a ticket where the nth digit is swapped with the *next* digit. 

> **Important:** We will be using these functions later, so don't write them to work just for airline tickets. Write them so they work for arbitrary lists of digits instead. 

> Note: If there are $N$ digits in a ticket, `n` should be between 0 and $n-1$ for `change-nth-digit-to` and between 0 and $n-2$ for `transpose-nth-digit`.

In [2]:
(defsnapshot todo-2)

(definec change-nth-digit-to (ticket :list-digits n :nat d :digit):list-digits
    (if (< (len ticket) (+ n 1))
        ticket
        (if (or (equal n 0) (endp ticket))
            (cons d (rest ticket))
            (append (list (first ticket)) (change-nth-digit-to (rest ticket) (- n 1) d)))))

(definec transpose-nth-digit (ticket :list-digits n :nat):list-digits
    (if (< (len ticket) (+ n 2))
        ticket
        (if (equal n 0)
            (cons (second ticket) (cons (first ticket) (rest(rest ticket))))
            (append (list (first ticket)) (transpose-nth-digit (rest ticket) (- n 1))))))


(check-expect (change-nth-digit-to '(1 2 3 4 5) 2 4) '(1 2 4 4 5))
(check-expect (change-nth-digit-to '(0 9 8) 2 4) '(0 9 4))
(check-expect (change-nth-digit-to '(1 2) 1 3) '(1 3))
(check-expect (change-nth-digit-to '(1 2 3 4 5 6 7 8 9 0) 8 4) '(1 2 3 4 5 6 7 8 4 0))
(check-expect (change-nth-digit-to '(9 4 2 5 6 7 8 3 4 5 6 7 2 1 3) 12 0) '(9 4 2 5 6 7 8 3 4 5 6 7 0 1 3))


(check-expect (transpose-nth-digit '(1 2 3 4 5) 2) '(1 2 4 3 5))
(check-expect (transpose-nth-digit '(1 2) 0) '(2 1))
(check-expect (transpose-nth-digit '(1 2 3 4 5 6 7 8 9 0) 2) '(1 2 4 3 5 6 7 8 9 0))
(check-expect (transpose-nth-digit '(9 4 2 5 6 7 8 3 4 5 6 7 2 1 3) 13) '(9 4 2 5 6 7 8 3 4 5 6 7 2 3 1))
(check-expect (transpose-nth-digit '(9 8 4 2 0) 0) '(8 9 4 2 0))

ACL2S !>>(DEFSNAPSHOT TODO-2)

Summary
Form:  ( DEFLABEL TODO-2 ...)
Rules: NIL
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
 TODO-2
ACL2S !>>(DEFINEC CHANGE-NTH-DIGIT-TO
                  (TICKET LIST-DIGITS N NAT D DIGIT)
                  LIST-DIGITS
                  (IF (< (LEN TICKET) (+ N 1))
                      TICKET
                      (IF (OR (EQUAL N 0) (ENDP TICKET))
                          (CONS D (REST TICKET))
                          (APPEND (LIST (FIRST TICKET))
                                  (CHANGE-NTH-DIGIT-TO (REST TICKET)
                                                       (- N 1)
                                                       D)))))

Form:  ( TEST-DEFINITION CHANGE-NTH-DIGIT-TO ... )
Form:  ( TEST-BODY-CONTRACTS CHANGE-NTH-DIGIT-TO... ) 
Form:  ( TEST-FUNCTION-CONTRACT CHANGE-NTH-DIGIT-TO ...) 
Testing: Done 
Elapsed Run Time: 2.34 seconds
Form:  ( ADMIT-DEFINITION CHANGE-NTH-DIGIT-TO ... )
Time:  0.03 seconds (prove: 0.00, pr

### TODO-3 (10 points)

Note that it is impossible for the checksum to be unique. There are only 10 different checksums, and there are far more than 10 possible ticket numbers. So we can't say that if we start with a valid ticket number, but somehow that number is garbled during transmission, we will definitely be able to detect that error. If we're unlucky (and it seems that is a 10% probability), we'll simply end up with another valid ticket number, even after a transmission failure. So the best we can hope for is that if *one* error happens during transmission, we should be able to detect that.

Let's focus on errors of the first type, i.e., when a single digit is randomly changed. Suppose that as start with a valid ticket $A = a_1 a_2 \dots a_{15}$ and end up with $B = a_1 a_2 \dots b_i \dots a_{15}$, where $a_i$ was replaced with $b_i$. We would like to show that ticket $B$ must be invalid. 

Use `test?` to show this property holds (but see next Note). Use `change-nth-digit-to` to create $B$ from an arbitrary ticket $A$, and keep in mind that the only valid index values `k` must be between 0 and 14, since airline tickets have precisely 15 digits.

> Note: Unfortunately, this scheme for validating airline ticket numbers isn't strong enough to catch all 1-digit errors. For example, if the first digit is changed from a 2 to a 9, this does not affect the ticket number modulo 7, so the same checksum will work! The best we can hope for here is that `test?` will show up some counterexamples that will help us understand the limitations of our scheme, and maybe come up with a different scheme if necessary.

> Note: Note also that you can get very "unlucky" in that you end up replacing the kth digit with a digit d that just happens to be the same as the old digit at the kth position. So your condition that you're trying to test should take this into account!

In [3]:
(defsnapshot todo-3)

(defdata ticketNum
    (range integer (0 <= _ <= 14)))

;; (definec get-nth-digit (ticket :list-digits n :nat):digit
;;     (if (< (len ticket) (+ n 1))
;;         'nil
;;         (if (equal n 0)
;;             (cons (second ticket) (cons (first ticket) (rest(rest ticket))))
;;             (append (list (first ticket)) (transpose-nth-digit (rest ticket) (- n 1))))))

(definec get-nthDigit (ticket :list-digits n :ticketNum):list-digits
    (if (< (- (len ticket) 1) n) 
        nil
        (if (equal n 0)
            (list (first ticket))
            (get-nthDigit (rest ticket) (- n 1)))))

(test? (implies (and (list-digitsp ticket) (natp nthDigit) (digitp newDigit) (< nthDigit (len ticket)) (equal (len ticket) 15)
                     (not (equal (get-nthDigit ticket nthDigit) newDigit))
                     (equal (valid-ticket ticket) t))
                (equal (valid-ticket (change-nth-digit-to ticket nthDigit newDigit)) nil)))
                
(check-expect (get-nthDigit '(1 2 5 4) 3) '(4))
(check-expect (get-nthDigit '(1 2 5 4 5 6 7 9 3 4 1) 9) '(4))
(check-expect (get-nthDigit '(1 2 5 4) 1) '(2))
(check-expect (get-nthDigit '(1 2 5 4) 0) '(1))
(check-expect (get-nthDigit '(1 2 5 4 3 4 3 0 2 8 3 9) 11) '(9))

ACL2S !>>(DEFSNAPSHOT TODO-3)

Summary
Form:  ( DEFLABEL TODO-3 ...)
Rules: NIL
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
 TODO-3
ACL2S !>>(DEFDATA TICKETNUM
                  (RANGE INTEGER (0 <= _ <= 14)))
 Predicate events...
Form:  ( DEFUN TICKETNUMP ...)
Form:  ( IN-THEORY (DISABLE* ...))
Form:  ( IN-THEORY (ENABLE ...))
Form:  ( TABLE ACL2::RULESET-TABLE ...)
Form:  ( MAKE-EVENT (LET* ...))
 Tau characterization events...
 (TICKETNUMP ACL2::V1) <= body -- not complete. 
Reasons: ("Illegal tau rule") 
 (TICKETNUMP ACL2::V1) => body -- not complete. 
Reasons: 
("The formula fails to fit any of the forms for acceptable :TAU-SYSTEM rules.")

Form:  ( DEFTHM TICKETNUM=>DEF ...)
 Enumerator events...
Form:  ( DEFUN NTH-TICKETNUM-BUILTIN ...)
Form:  ( DEFUN NTH-TICKETNUM/ACC-BUILTIN ...)
Form:  ( PROGN (SET-BOGUS-DEFUN-HINTS-OK T) ...)
Form:  ( ENCAPSULATE NIL (LOGIC) ...)
Time:  0.09 seconds (prove: 0.01, print: 0.00, other: 0.08)
 Registering type...
Form:  ( DEFUN N

### TODO-4 (10 points)

Now let's try the srecond type of error. Again, if we start with a valid ticket $A = a_1 a_2 \dots a_{15}$ then transpose two digits to end up with ticket $B = a_1 a_2 \dots a_{i+1} a_i \dots a_{15}$, we would like to believe that ticket $B$ is invalid. Use `test?` to verify this property.  Your answer should be very similar to TODO-3.

> Note: Again, this conjecture is not true, so the real exercise is to see if the randomized testing helps us to understand the true properties of our code. In fact, suppose we transpose $a_i$ and ${i+1}$. Using properties of modulos (make sure you understand why), we see that $B$ will have the same checksum as $A$ if $10a_i + a_{i+1} \equiv 10a_{i+1} + a_i \pmod 7$. With a little algebra (make sure you know how), we can show that this is true precisely when $a_i \equiv b_i \pmod 7$, e.g., when we're transposing 07, 18, 29, 70, 81, or 92. In other words, the checksum is valid 6% of the time!

In [4]:
(defsnapshot todo-4)



(test? (implies (and (list-digitsp ticket) (natp nthDigit) (< nthDigit (len ticket)) (equal (len ticket) 15)
                     (equal (valid-ticket ticket) t))
                (equal (valid-ticket (transpose-nth-digit ticket nthDigit)) nil)))

ACL2S !>>(DEFSNAPSHOT TODO-4)

Summary
Form:  ( DEFLABEL TODO-4 ...)
Rules: NIL
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
 TODO-4
ACL2S !>>(TEST?
             (IMPLIES
                  (AND (LIST-DIGITSP TICKET)
                       (NATP NTHDIGIT)
                       (< NTHDIGIT (LEN TICKET))
                       (EQUAL (LEN TICKET) 15)
                       (EQUAL (VALID-TICKET TICKET) T))
                  (EQUAL (VALID-TICKET (TRANSPOSE-NTH-DIGIT TICKET NTHDIGIT))
                         NIL)))

**Summary of Cgen/testing**
We tested 3000 examples across 3 subgoals, of which 0 (0 unique) satisfied
the hypotheses, and found 0 counterexamples and 0 witnesses.

Test? succeeded. No counterexamples were found.

## Bank Routing Numbers

The airline ticket checksum relies on converting a list of 14 digits into a single number, then doing modular arithmetic on that number. Bank routing numbers use a different scheme.

Firat, bank routing numbers are nine-digit numbers. There is no "checksum" per se, but the numbers are chosen such that $A = a_1 a_2 \dots a_9$ is valid precisely when
$$7a_1 + 3a_2 + 9a_3 + 7a_4 + 3a_5 + 9a_6 + 7a_7 + 3a_8 + 9a_9 \equiv 0 \pmod {10}$$
The key fact here is that the digits are weighted.

> Aside: With airline ticket numbers, the digit a_{15} was the remainder of something to do with the first 14 digits, but with bank routing numbers, there is a general equation involving all 9 digits. This distinction is superficial. I.e., we could say that $a_9 \euiv -(\cdots) \pmod {10}$ where $(\cdots)$ is the weighted sum of the first eight digits. Also, in airline ticket numbers we computed a large number from the first 14 digits, whereas with bank routing numbers we are adding the digits but with different weights depending on the position. In fact, the weights idea is more general. Converting the digits to numbers is the same as using weights that are the powers of 10! So you see, the idea behind routing numbers is a generalization of the idea behind airline ticket numbers!

### TODO-5 (10 points)

This idea of using weights is very powerful, and many other checksum ideas use it. So let's start with a function that takes care of the weights. You may recognize the relevant operation as the dot product of two vectors:
$$\langle a_1, a_2, \dots, a_n\rangle \cdot \langle w_1, w_2, \dots, w_n\rangle = \sum_{i=1}^{n} a_i \times w_i$$

Define the function `(dotpr as ws)` in ACL2.


In [5]:
(defsnapshot todo-5)



(definec dotpr (as :list-digits ws :list-digits):nat
    (if (or (endp as) (endp ws))
        0
        (+ (* (first as) (first ws)) (dotpr (rest as) (rest ws)))))

(check-expect (dotpr '(0 1) '(9 0)) 0)
(check-expect (dotpr '(0 1 1) '(9 0 9)) 9)
(check-expect (dotpr '(2 1) '(2 1)) 5)
(check-expect (dotpr '(3 1 1) '(3 1 1)) 11)
(check-expect (dotpr '(3 2 4) '(1 1 1)) 9)

ACL2S !>>(DEFSNAPSHOT TODO-5)

Summary
Form:  ( DEFLABEL TODO-5 ...)
Rules: NIL
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
 TODO-5
ACL2S !>>(DEFINEC DOTPR (AS LIST-DIGITS WS LIST-DIGITS)
                  NAT
                  (IF (OR (ENDP AS) (ENDP WS))
                      0
                      (+ (* (FIRST AS) (FIRST WS))
                         (DOTPR (REST AS) (REST WS)))))

Form:  ( TEST-DEFINITION DOTPR ... )
Form:  ( TEST-BODY-CONTRACTS DOTPR... ) 
Form:  ( TEST-FUNCTION-CONTRACT DOTPR ...) 
Testing: Done 
Elapsed Run Time: 1.22 seconds
Form:  ( ADMIT-DEFINITION DOTPR ... )
Time:  0.02 seconds (prove: 0.00, print: 0.00, other: 0.02)
Form:  ( PROVE-FUNCTION-CONTRACT DOTPR ... )
Time:  0.14 seconds (prove: 0.07, print: 0.00, other: 0.06)
Form:  ( PROVE-BODY-CONTRACTS DOTPR ... )
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
Elapsed Run Time: 0.21 seconds
Function Name : DOTPR 
Termination proven -------- [*] 
Function Contract proven -- [*] 
Bo

### TODO-6 (10 points)

Now use your definition of `dotpr` to define `valid-routing-number`. For example,

    (valid-routing-number '(1 2 2 1 5 0 2 7 8)) = T


In [6]:
(defsnapshot todo-6)


(definec valid-routing-number (routingNum :list-digits):bool
    (if (equal (len routingNum) 9)
        (if (equal (mod (dotpr routingNum '(7 3 9 7 3 9 7 3 9)) 10) 0)
            t
            nil)
        nil))



(check-expect (valid-routing-number '(1 2 2 1 5 0 2 7 8)) t)
(check-expect (valid-routing-number '(1 1 0 0 0 0 0 0 0)) t)
(check-expect (valid-routing-number '(1 2 2 1 5 8 2 7 0)) t)
(check-expect (valid-routing-number '(1 2 3 1 5 0 2 7 8)) nil)
(check-expect (valid-routing-number '(2 5 8 1 2 0 1 7 2)) t)

ACL2S !>>(DEFSNAPSHOT TODO-6)

Summary
Form:  ( DEFLABEL TODO-6 ...)
Rules: NIL
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
 TODO-6
ACL2S !>>(DEFINEC VALID-ROUTING-NUMBER
                  (ROUTINGNUM LIST-DIGITS)
                  BOOL
                  (IF (EQUAL (LEN ROUTINGNUM) 9)
                      (IF (EQUAL (MOD (DOTPR ROUTINGNUM '(7 3 9 7 3 9 7 3 9))
                                      10)
                                 0)
                          T NIL)
                      NIL))

Form:  ( TEST-DEFINITION VALID-ROUTING-NUMBER ... )
Form:  ( TEST-BODY-CONTRACTS VALID-ROUTING-NUMBER... ) 
Form:  ( TEST-FUNCTION-CONTRACT VALID-ROUTING-NUMBER ...) 
Testing: Done 
Elapsed Run Time: 0.20 seconds
Form:  ( ADMIT-DEFINITION VALID-ROUTING-NUMBER ... )
Time:  0.02 seconds (prove: 0.00, print: 0.00, other: 0.02)
Form:  ( PROVE-FUNCTION-CONTRACT VALID-ROUTING-NUMBER ... )
Time:  0.10 seconds (prove: 0.03, print: 0.00, other: 0.07)
Form:  ( PROVE-BODY-CONTRACTS VALI

### TODO-7 (10 points)

This should be very similar to TODO-3. Use `test?` to show that if you have a valid routing number, then change a digit in the number, the result is **not** a valid routing number.

In this case, you should not see any counterexamples. Here's why. Suppose we change the first digit, so that $a_1\ne b_1$. Then the sum of the other digits is completely unaffected, so the total sum changes only by $7a_1 - 7b_1 \bmod 10$, where the "7" comes from the weight in the first position. In other words, the checksum does \emph{not} change precisely when $7a_1 \equiv 7b_1 \pmod{10}$. When is that? Since 7 and 10 are relatively prime, this only happens when $a_1 \equiv b_1 \pmod{10}$. (For an explanation, you can multiply both sides of the first equation by $7^{-1} \bmod 10$, and that exists because 7 and 10 are relatively prime. In particular, $3\times7 =21 \equiv 1 \pmod{10}$, so $7^{-1} \bmod 10 = 3$.) So if $a_1$ is changed, the checksum has to change, too. That means that a singled-digit error will *always* be detected. So if you see some counterexamples, there is a bug in your code!

In principle, you could prove this in ACL2 by formalizing the argument in the previous paragraph (and generalizing it to all three different weights.) But that is so much work that I won't ask you to do it, even for extra credit. If you do want to pursue such proofs in ACL2, come see me about undergraduate research. Seriously, I have an idea that could turn this projecty into a nice publication for a student.

> Note: The reason this works is that the weights and the modulo are relatively prime. Since the modulo is 10, this means that the weights 2, 4, 5, 6, and 8 are all bad. For example, if the weight is even, then the checksum will be the same if a 3 is changed to an 8.

In [7]:
(defsnapshot todo-7)

(test? (implies (and (list-digitsp routNum) (natp nthDigit) (digitp newDigit) (< nthDigit (len routNum)) (equal (len routNum) 9)
                     (not (equal (get-nthDigit routNum nthDigit) newDigit))
                     (equal (valid-routing-number routNum) t))
                (equal (valid-routing-number (change-nth-digit-to routNum nthDigit newDigit)) nil)))

ACL2S !>>(DEFSNAPSHOT TODO-7)

Summary
Form:  ( DEFLABEL TODO-7 ...)
Rules: NIL
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
 TODO-7
ACL2S !>>(TEST?
            (IMPLIES
                 (AND (LIST-DIGITSP ROUTNUM)
                      (NATP NTHDIGIT)
                      (DIGITP NEWDIGIT)
                      (< NTHDIGIT (LEN ROUTNUM))
                      (EQUAL (LEN ROUTNUM) 9)
                      (NOT (EQUAL (GET-NTHDIGIT ROUTNUM NTHDIGIT)
                                  NEWDIGIT))
                      (EQUAL (VALID-ROUTING-NUMBER ROUTNUM)
                             T))
                 (EQUAL (VALID-ROUTING-NUMBER
                             (CHANGE-NTH-DIGIT-TO ROUTNUM NTHDIGIT NEWDIGIT))
                        NIL)))

**Summary of Cgen/testing**
We tested 3000 examples across 3 subgoals, of which 0 (0 unique) satisfied
the hypotheses, and found 0 counterexamples and 0 witnesses.

Test? succeeded. No counterexamples were found.

### TODO-8 (10 points)

This should be very similar to TODO-4. Use `test?` to show that if you have a valid routing number, then trnspose two adjacent digits in the number, the result is **not** a valid routing number.

Should you expect any counterexamples? Think of the difference between $7a_1+3a_2$ and $7a_2+3a_1$ mod 10? Well, suppose $(7a_1+3a_2) - (7a_2+3a_1) \equiv 0 \pmod {10}$. With some algebra, you can see that this is the same as
$(7-3)(a_1-a_2) = 4(a_1-a_2) \equiv 0 \pmod {10}$. Since 4 is even, this happens exactly when $a_1-a_2$ is a
multiple of 5, e.g., when $a_1=3$ and $a_2=8$, or about 10% of the time. Again, the question is whether randomized testing lets us see the properties of our code.

> Note: This same argument works whenever the difference between adjacent weights is even. In those cases, there is no way to detect a transposition of digits that differ by 5.



In [23]:
(defsnapshot todo-8)

(test? (implies (and (list-digitsp routNum) (natp nthDigit) (< nthDigit (len routNum)) (equal (len routNum) 9)
                     (not (equal (get-nthDigit routNum nthDigit) newDigit))
                     (equal (valid-routing-number routNum) t))
                (equal (valid-routing-number (transpose-nth-digit routNum nthDigit)) nil)))

ACL2S !>>(DEFSNAPSHOT TODO-8)
          38:x(DEFSNAPSHOT TODO-7)

Summary
Form:  ( DEFLABEL TODO-8 ...)
Rules: NIL
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
 TODO-8
ACL2S !>>(TEST?
          (IMPLIES
           (AND (LIST-DIGITSP ROUTNUM)
                (NATP NTHDIGIT)
                (< NTHDIGIT (LEN ROUTNUM))
                (EQUAL (LEN ROUTNUM) 9)
                (NOT (EQUAL (GET-NTHDIGIT ROUTNUM NTHDIGIT)
                            NEWDIGIT))
                (EQUAL (VALID-ROUTING-NUMBER ROUTNUM)
                       T))
           (EQUAL
                (VALID-ROUTING-NUMBER (TRANSPOSE-NTH-DIGIT ROUTNUM NTHDIGIT))
                NIL)))

**Summary of Cgen/testing**
We tested 3000 examples across 3 subgoals, of which 0 (0 unique) satisfied
the hypotheses, and found 0 counterexamples and 0 witnesses.

Test? succeeded. No counterexamples were found.

## Credit Cards

Many credit cards use the following checksum scheme. The credit card number is 16 digits, $A=a_1 a_2 \dots a_{16}$ and these are summed using weights, as before. The weights are $2, 1, 2, 1, \dots, 2, 1$. As before, the last digit is chosen so that the weighted sum (almost) is congruent to 0 modulo 10. But there's a twist. It's not just the sum that matters, it's actually
$$2a_1 + a_2 + 2a_3 + a_4 + \dots + 2a_{15} + a_{16} + \text{number of $a_{\text{odd } i}$ that are $\ge 5$}$$
What a weird twist! The number of $a_i$ (where $i$ is odd) that are $\ge 5$. For example, consider the credit card number $4000~0012~3456~7899$. The $a_i$ for odd $i$ are $40~01~35~79$, and the number of those that are $\ge 5$ is 3.

> Aside: If you took COSC 1010 at UW, you may remember this seemingly crazy validation scheme. Implementing this scheme in Java was a favorite exercise of the recently retired but long-serving COSC 1010 instructor at UW! First-year students can write the validation code, though they probably never guessed *why* it was defined this way. As you can probably guess by now, the reason has to do with modular arithmetic!

So why this crazy scheme? As we saw earlier, using weights that are even means that a single digit $a_i$ can be replaced by another digit $b_i$ such that $|a_i-b_i|=5$ and this change goes undetected modulo 10. But, that's where the "$\ge 5$" component comes in. If $|a_i-b_i|=5$, precisely one of them is $\ge 5$, so if one is substituted for the other, the number of digits $\ge 5$ changes by 1. And notice that we only count the digits in odd locations, *which are precisely the digits that have an even weight (i.e., 2).* That's why this scheme works to detect single-digit errors, even though we use even weights.

But what about a transposition error? Suppose that $a_1$ and $a_2$ are swapped. Then the weight will be the same if $(2a_1 + a_2) - (2a_2 + a_1) = a_1 - a_2 \equiv 0 \pmod {10}$, which means that we can see a difference modulo 10 whenever $a_1 \ne a_2$. But what about the $\ge 5$ count? If $a_1 \not\ge 5$ but $a_2 \ge 5$, then the count of digits in odd positions will increase by one. Can that offset the change of the digit sum modulo 10? Well, yes. The count increased by 1, so to offset it the sum needs to be $(2a_1 + a_2) - (2a_2 + a_1) = a_1 - a_2 \equiv -1 \pmod {10}$. Remember that $-1 \bmod 10 = 9$, so this means that an adjacent $90$ can be swapped with $09$ and be undetected.

This crazy checksum scheme is successful in catching all single-digit errors *and* almost all (98%) transpositions!

### TODO-9 (10 points)

Define `valid-ccard-number`, just like in TODO-6. For example,

    (valid-ccard-number '(4 0 0 0 0 0 1 2 3 4 5 6 7 8 9 9)) = t

In [9]:
(defsnapshot todo-9)

(definec findOdds (ccard :list-digits index :nat):nat
    (if (endp ccard)
        0
        (if (and (equal (mod index 2) 1) (>= (first ccard) 5))
            (+ 1 (findOdds (rest ccard) (+ index 1)))
            (findOdds (rest ccard) (+ index 1)))))

(check-expect (findOdds '(1 2 1 2) 1) 0)
(check-expect (findOdds '(1 2 1 2 7) 1) 1)
(check-expect (findOdds '(1 2 1 2 5 2 5 1) 1) 2)
(check-expect (findOdds '(5 3 6 7 9 4 9) 1) 4)
(check-expect (findOdds '(9) 1) 1)

(definec valid-ccard-number (ccard :list-digits):bool
    (if (not (equal (len ccard) 16))
        nil
        (if (equal (mod (+ (dotpr ccard '(2 1 2 1 2 1 2 1 2 1 2 1 2 1 2 1)) (findOdds ccard 1)) 10) 0)
            t
            nil)))

(check-expect (valid-ccard-number '(4 0 0 0 0 0 1 2 3 4 5 6 7 8 9 9)) t)
(check-expect (valid-ccard-number '(1 0 1 0 1 0 1 0 1 0 0 0 0 0 0 0)) t)
(check-expect (valid-ccard-number '(0 0 4 0 0 0 1 2 3 4 5 6 7 9 9 8)) t)
(check-expect (valid-ccard-number '(9 0 0 0 0 0 3 2 1 4 5 6 7 8 4 9)) t)
(check-expect (valid-ccard-number '(4 0 0 0 0 0 1 2 0 4 5 6 7 8 9 9)) nil)

ACL2S !>>(DEFSNAPSHOT TODO-9)

Summary
Form:  ( DEFLABEL TODO-9 ...)
Rules: NIL
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
 TODO-9
ACL2S !>>(DEFINEC FINDODDS (CCARD LIST-DIGITS INDEX NAT)
                  NAT
                  (IF (ENDP CCARD)
                      0
                      (IF (AND (EQUAL (MOD INDEX 2) 1)
                               (>= (FIRST CCARD) 5))
                          (+ 1 (FINDODDS (REST CCARD) (+ INDEX 1)))
                          (FINDODDS (REST CCARD) (+ INDEX 1)))))

Form:  ( TEST-DEFINITION FINDODDS ... )
Form:  ( TEST-BODY-CONTRACTS FINDODDS... ) 
Form:  ( TEST-FUNCTION-CONTRACT FINDODDS ...) 
Testing: Done 
Elapsed Run Time: 1.85 seconds
Form:  ( ADMIT-DEFINITION FINDODDS ... )
Time:  0.21 seconds (prove: 0.00, print: 0.00, other: 0.21)
Form:  ( PROVE-FUNCTION-CONTRACT FINDODDS ... )
Time:  0.10 seconds (prove: 0.04, print: 0.00, other: 0.06)
Form:  ( PROVE-BODY-CONTRACTS FINDODDS ... )
Time:  0.02 seconds (prove: 0.00, print: 

### TODO-10 (10 points)

This should be very similar to TODO-7. Use `test?` to show that if you have a valid credit card number, then change a digit in the number, the result is **not** a valid credit card number. As we discussed above, you should see no counterexamples. I am not asking you to prove this, just to validate it using random testing with `test?`.

In [10]:
(defsnapshot todo-10)

(test? (implies (and (list-digitsp ccard) (natp nthDigit) (digitp newDigit) (< nthDigit (len ccard)) (equal (len ccard) 16)
                     (not (equal (get-nthDigit ccard nthDigit) newDigit))
                     (equal (valid-ccard-number ccard) t))
                (equal (valid-ccard-number (change-nth-digit-to ccard nthDigit newDigit)) nil)))


ACL2S !>>(DEFSNAPSHOT TODO-10)

Summary
Form:  ( DEFLABEL TODO-10 ...)
Rules: NIL
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
 TODO-10
ACL2S !>>(TEST?
          (IMPLIES (AND (LIST-DIGITSP CCARD)
                        (NATP NTHDIGIT)
                        (DIGITP NEWDIGIT)
                        (< NTHDIGIT (LEN CCARD))
                        (EQUAL (LEN CCARD) 16)
                        (NOT (EQUAL (GET-NTHDIGIT CCARD NTHDIGIT)
                                    NEWDIGIT))
                        (EQUAL (VALID-CCARD-NUMBER CCARD) T))
                   (EQUAL (VALID-CCARD-NUMBER
                               (CHANGE-NTH-DIGIT-TO CCARD NTHDIGIT NEWDIGIT))
                          NIL)))

**Summary of Cgen/testing**
We tested 6000 examples across 6 subgoals, of which 0 (0 unique) satisfied
the hypotheses, and found 0 counterexamples and 0 witnesses.

Test? succeeded. No counterexamples were found.

### TODO-11 (10 points)

This should be very similar to TODO-8. Use `test?` to show that if you have a valid credit card number, then transpose two adjacent digits in the number, the result is not a valid credit card number. As discussed above you should only see counterexamples when the two numbers transposed are 09 or 90.

In [11]:
(defsnapshot todo-11)

(test? (implies (and (list-digitsp ccard) (natp nthDigit) (< nthDigit (len ccard)) (equal (len ccard) 16)
                     (equal (valid-ccard-number ccard) t))
                (equal (valid-ccard-number (transpose-nth-digit ccard nthDigit)) nil)))

ACL2S !>>(DEFSNAPSHOT TODO-11)

Summary
Form:  ( DEFLABEL TODO-11 ...)
Rules: NIL
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
 TODO-11
ACL2S !>>(TEST?
          (IMPLIES
             (AND (LIST-DIGITSP CCARD)
                  (NATP NTHDIGIT)
                  (< NTHDIGIT (LEN CCARD))
                  (EQUAL (LEN CCARD) 16)
                  (EQUAL (VALID-CCARD-NUMBER CCARD) T))
             (EQUAL (VALID-CCARD-NUMBER (TRANSPOSE-NTH-DIGIT CCARD NTHDIGIT))
                    NIL)))

**Summary of Cgen/testing**
We tested 6000 examples across 6 subgoals, of which 0 (0 unique) satisfied
the hypotheses, and found 0 counterexamples and 0 witnesses.

Test? succeeded. No counterexamples were found.

## ISBNs: Full Error Detection

As we've seen in the past two examples, the "weighted sum mod 10" approach requires that

* weights must not be even or 5 
* adjacent weights differ by an odd amount other than 5.

In both cases, the problem is that weights and adjacent differences must be **relatively prime** to 10, and that is so that $a^{-1} \pmod {10}$ exists.

But you know of another way of ensuring that the weights and adjacent differences are relatively prime? Stop using modulo 10, and instead use modulo a **prime number** $N$. Then all weights and differences are (by definition) relatively prime to $N$. And it just so happens that $11$ is a prime number that's already close to 10!

ISBN numbers (i.e., identifiers for books) follow this approach. An ISBN has 10 digits $A = a_1 a_2 \dots a_{10}$ and the tenth digit is chosen so that the weighted sum with the weights $\langle 10, 9, 8, \dots, 1\rangle$ is equal to $0\pmod{11}$. Since we're doing modulo 11, it is possible that the last "digit" must be 10, so the ISBN format allows the last digit to be 0-9 or X. For example, my favorite book of all time has ISBN 198480278X.

> Aside: There is a newer ISBN-13 format used for books and other media, but the ISBN code we're describing above is the older ISBN-10. It happens that ISBN-13 doesn't have as good error detection properties, because it went back to "mod 10".

### TODO-12 (10 points)

Define the function `(valid-isbn isbn)` that checks to see if an ISBN is actually valid. You will also have to define the data type `isbn`, which consists of a list of 9 digits and a "digit-or-X". For example,

    (valid-isbn '(1 9 8 4 8 0 2 7 8 X)) = t

In [12]:
(defsnapshot todo-12)

(defdata x-or-digit
    (oneof digit
           'x))

(defdata isbn
    (listof x-or-digit))

(definec dotprISBN (as :isbn ws :nat-list):nat
    (if (or (endp as) (endp ws))
        0
        (if (equal (first as) 'X)
            (* 10 (first ws))
            (+ (* (first as) (first ws)) (dotprISBN (rest as) (rest ws))))))

(check-expect (dotprISBN '(1 2 X) '(1 2 3)) 35)
(check-expect (dotprISBN '(1 9 X) '(1 1 1)) 20)
(check-expect (dotprISBN '(X) '(10)) 100)
(check-expect (dotprISBN '(1 9) '(1 1)) 10)
(check-expect (dotprISBN '(1 8 1) '(1 1 1)) 10)

(definec checkForX (isbn :isbn):bool
    (if (or (endp isbn) (equal (len isbn) 1))
        t
        (if (equal (first isbn) 'X)
            nil
            (checkForX (rest isbn)))))

(check-expect (checkForX '(1 9 8 4 8 0 2 7 8 X)) T)
(check-expect (checkForX '(1 9 1 0 0 0 0 0 0 0)) T)
(check-expect (checkForX '(1 9 X 4 8 0 2 7 8 X)) NIL)
(check-expect (checkForX '(1 9 8 4 X 0 2 7 8 X)) NIL)
(check-expect (checkForX '(1 9 8 4 8 0 2 7 8 0)) T)


(definec valid-isbn (isbn :isbn):bool
    (if (equal (len isbn) 10)
        (if (checkForX isbn)
            (equal (mod (dotprISBN isbn '(10 9 8 7 6 5 4 3 2 1)) 11) 0)
            nil)
        nil))

(check-expect (valid-isbn '(1 9 8 4 8 0 2 7 8 X)) T)
(check-expect (valid-isbn '(1 9 1 0 0 0 0 0 0 0)) T)
(check-expect (valid-isbn '(1 X)) NIL)
(check-expect (valid-isbn '(1 1 1 1 0 0 1 1 9 9)) NIL)
(check-expect (valid-isbn '(1 X 0 0 9 0 0 0 1 0)) NIL)

ACL2S !>>(DEFSNAPSHOT TODO-12)

Summary
Form:  ( DEFLABEL TODO-12 ...)
Rules: NIL
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
 TODO-12
ACL2S !>>(DEFDATA X-OR-DIGIT (ONEOF DIGIT 'X))
 Predicate events...
Form:  ( DEFUN X-OR-DIGITP ...)
Form:  ( IN-THEORY (DISABLE* ...))
Form:  ( IN-THEORY (ENABLE ...))
Form:  ( TABLE ACL2::RULESET-TABLE ...)
Form:  ( MAKE-EVENT (LET* ...))
 Tau characterization events...
Defdata/Note: X-OR-DIGITP relatively complete for Tau.
Form:  ( DEFTHM DEF=>X-OR-DIGIT ...)
Form:  ( DEFTHM X-OR-DIGIT=>DEF ...)
 Enumerator events...
Form:  ( DEFUN NTH-X-OR-DIGIT-BUILTIN ...)
Form:  ( DEFUN NTH-X-OR-DIGIT/ACC-BUILTIN ...)
Form:  ( PROGN (SET-BOGUS-DEFUN-HINTS-OK T) ...)
Form:  ( ENCAPSULATE NIL (LOGIC) ...)
Time:  0.06 seconds (prove: 0.02, print: 0.00, other: 0.04)
 Registering type...
Form:  ( DEFUN NTH-X-OR-DIGIT ...)
Form:  ( ENCAPSULATE (((NTH-X-OR-DIGIT * ...) ...) ...) ...)
Form:  ( DEFUN NTH-X-OR-DIGIT/ACC ...)
Form:  ( ENCAPSULATE (((NTH-X-OR-

### TODO-13 (10 points)


This should be very similar to TODO-7. Use test? to show that if you have a valid isbn, then change a digit in the isbn, the result is not a valid isbn. As we discussed above, you should see no counterexamples, since we are using "mod 11" so all weights are relatively prime to the modulus. I am not asking you to prove this, just to validate it using random testing with `test?`.


In [13]:
(defsnapshot todo-13)

(test? (implies (and (isbnp isbn) (natp nthDigit) (x-or-digitp newDigit) (< nthDigit (len isbn)) (equal (len isbn) 10)
                     (not (equal (get-nthDigit isbn nthDigit) newDigit))
                     (equal (valid-isbn isbn) t))
                (equal (valid-isbn (change-nth-digit-to isbn nthDigit newDigit)) nil)))



ACL2S !>>(DEFSNAPSHOT TODO-13)

Summary
Form:  ( DEFLABEL TODO-13 ...)
Rules: NIL
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
 TODO-13
ACL2S !>>(TEST?
          (IMPLIES
             (AND (ISBNP ISBN)
                  (NATP NTHDIGIT)
                  (X-OR-DIGITP NEWDIGIT)
                  (< NTHDIGIT (LEN ISBN))
                  (EQUAL (LEN ISBN) 10)
                  (NOT (EQUAL (GET-NTHDIGIT ISBN NTHDIGIT)
                              NEWDIGIT))
                  (EQUAL (VALID-ISBN ISBN) T))
             (EQUAL (VALID-ISBN (CHANGE-NTH-DIGIT-TO ISBN NTHDIGIT NEWDIGIT))
                    NIL)))

**Summary of Cgen/testing**
We tested 4000 examples across 4 subgoals, of which 0 (0 unique) satisfied
the hypotheses, and found 0 counterexamples and 0 witnesses.

Test? succeeded. No counterexamples were found.

### TODO-14 (10 points)

This should be very similar to TODO-8. Use `test?` to show that if you have a valid isbn, then transpose two adjacent digits in the isbn, the result is not a valid isbn. As discussed above you should not see any counterexamples since we're using "mod 11" which is prime. I am not asking you to prove this, just to validate it using random testing with `test?`.

In [14]:
(defsnapshot todo-14)


(test? (implies (and (isbnp isbn) (natp nthDigit) (x-or-digitp newDigit) (< nthDigit (len isbn)) (equal (len isbn) 10)
                     (equal (valid-isbn isbn) t))
                (equal (valid-isbn (transpose-nth-digit isbn nthDigit)) nil)))

ACL2S !>>(DEFSNAPSHOT TODO-14)

Summary
Form:  ( DEFLABEL TODO-14 ...)
Rules: NIL
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
 TODO-14
ACL2S !>>(TEST?
             (IMPLIES (AND (ISBNP ISBN)
                           (NATP NTHDIGIT)
                           (X-OR-DIGITP NEWDIGIT)
                           (< NTHDIGIT (LEN ISBN))
                           (EQUAL (LEN ISBN) 10)
                           (EQUAL (VALID-ISBN ISBN) T))
                      (EQUAL (VALID-ISBN (TRANSPOSE-NTH-DIGIT ISBN NTHDIGIT))
                             NIL)))

**Summary of Cgen/testing**
We tested 4000 examples across 4 subgoals, of which 0 (0 unique) satisfied
the hypotheses, and found 0 counterexamples and 0 witnesses.

Test? succeeded. No counterexamples were found.

## Postal Barcodes: Error Detection *and* Correction

We've seen how we can detect all one-digit and transposition errors using a weighted sum modulo a prime number, and the cost is simply having an extra digit in the message. If we allow even more overhead (i.e., a larger checksum), we can detect **and correct** these errors.

The code we're going to explore is from the post office. You may have gotten mail with a yellow sticker attached, and in that sticker is a barcode with long and short lines&mdash;a list of ones and zeros, if you will. These are used to encode important information in a machine-readable format. In particular, the barcode has four parts:

1. The Zip code
2. The extended 4-digit portion of the zip code
3. a 2-digit code for presorting mail, e.g., a portion of the house number
4. a checksum to make the sunm of all 12 digits equal to 0 modulo 10

For example, a code to my home address may look like `82063 9201 18 0`.

Wait a minute. This system is using "mod 10" and doesn't have any weights, so how can it be able to detect and correct errors?

The answer is that the digits themselves are encoded into binary. So instead of having a list of 12 digits, it is actually a list of 60 bits. The encoding of digits to bits is as follows:


| Digit   | Bits   | Digit   | Bits   | Digit   | Bits   | Digit   | Bits   | Digit   | Bits   |
| ------- | ------ | ------- | ------ | ------- | ------ | ------- | ------ | ------- | ------ |
| 0       | 11000  | 1       | 00011  | 2       | 00101  | 3       | 00110  | 4       | 01001  |
| 5       | 01010  | 6       | 01100  | 7       | 10001  | 8       | 10010  | 9       | 10100  |

Notice we are using five bits to encode each digit, even though only four bits are necessary. That is the extra overhead we're paying to get error detection and correction.

How does this work? First, notice that all 5-bit code have three 0s and two 1s. If you are writing this code in December, then you can answer the question, "How many ways can you write down exactly three 0s and two 1s?". (Hint: It's 5!/3!2! = 10.) So there is a unique code for each digit.

Now, suppose a single bit flips from 0 to 1 or 1 to 0. Then we can tell exactly which digit is encoded incorrectly, since that digit will no longer have exactly three 0s and two 1s. Suppose we know that the first digit has an error. Then we can use the "mod 10" checksum to find what that digit must be! (Realize that the weights are all 1s, which are relatively prime to 10, so single errors in the **digits** can be detected.) Since we know what the right digit should have been, it's easy to replace the five bits that correspond to that digit with the correct five bits.

What about transposition errors? Suppose a transposition occurs within the five bits for a given digit. Then the digit will change to another *valid* digit. (Remember, there are exactly ten bit patterns with threee 0s and two 1s, so swapping two bits will result in another one of those ten bit patterns.) The "mod 10" checksum will detect that there's an error, but there is no way to discover which of the digits is wrong. So this error is not corrected. On the other hand, if a transposition occurs at the boundary of two digits, then both digits will have invalid 5-bit patterns. That means we know which two bits were transposed (at the boundary of the two illegal digits), and we can swap them back to correct the error.

So this scheme detects all one-bit and transposition errors, and corrects all one-bit errors and many transposition errors. Since this is intended for computer communication, one-bit errors are common but transposition errors are not. (Transposition errors are common to humans&mdash;just see all the typos in this assignment, for example. But computers just don't make those kind of mistakes. On the other hand, computers are more likely to have a one-bit error during transmission, due to faulty wiring, cosmic rays, disk failures, etc.) So this is a very effective error detection and (mostly) correction scheme! 


### TODO-15 (10 points)

Define the functions `(encode digits)` and `(decode bits)` that convert a list of digits to a list of 5-bit words and vice versa. For example,

    (encode '(8 2 0 6 3)) = '((1 0 0 1 0) (0 0 1 0 1) (1 1 0 0 0) (0 1 1 0 0) (0 0 1 1 0))
    (decode '((1 0 0 1 0) (0 0 1 0 1) (1 1 0 0 0) (0 1 1 0 0) (0 0 1 1 0))) = '(8 2 0 6 3)

In preparation for the rest of this assignment, have `decode` return the digit 0 for any 5-bit word that it does not recognize.

Of course, you will have to define any relevant types with `defdata`.

> Note: I chose to store the bits as a list of words to make it easier to write the encode and decode functions. It really should be a list of bits!

> Hint: There are more clever ways of doing this, but I just used a (deeply) nested if-statement to convert from individual digits to words and vice versa.


In [15]:
(defsnapshot todo-15)

(defdata bit
    (oneof 0
           1))

(defdata bits
    (list bit
          bit
          bit
          bit
          bit))

(defdata bit-list
    (listof bits))

(definec encode (digits :list-digits):bit-list
    (if (endp digits)
        'nil
        (if (equal (first digits) 0)
            (append (list (list 1 1 0 0 0)) (encode (rest digits)))
        (if (equal (first digits) 1)
            (append (list (list 0 0 0 1 1)) (encode (rest digits)))
        (if (equal (first digits) 2)
            (append (list (list 0 0 1 0 1)) (encode (rest digits)))
        (if (equal (first digits) 3)
            (append (list (list 0 0 1 1 0)) (encode (rest digits)))
        (if (equal (first digits) 4)
            (append (list (list 0 1 0 0 1)) (encode (rest digits)))
        (if (equal (first digits) 5)
            (append (list (list 0 1 0 1 0)) (encode (rest digits)))
        (if (equal (first digits) 6)
            (append (list (list 0 1 1 0 0)) (encode (rest digits)))
        (if (equal (first digits) 7)
            (append (list (list 1 0 0 0 1)) (encode (rest digits)))
        (if (equal (first digits) 8)
            (append (list (list 1 0 0 1 0)) (encode (rest digits)))
            (append (list (list 1 0 1 0 0)) (encode (rest digits))))))))))))))

(check-expect (encode '(1 2)) '((0 0 0 1 1)(0 0 1 0 1)))
(check-expect (encode 'nil) 'nil)
(check-expect (encode '(8 2 0 6 3)) '((1 0 0 1 0) (0 0 1 0 1) (1 1 0 0 0) (0 1 1 0 0) (0 0 1 1 0)))
(check-expect (encode '(3)) '((0 0 1 1 0)))
(check-expect (encode '(9 9 9 9 9)) '((1 0 1 0 0)(1 0 1 0 0)(1 0 1 0 0)(1 0 1 0 0)(1 0 1 0 0)))

(definec decode (bits :bit-list):list-digits
    (if (endp bits)
        'nil
        (if (equal (first bits) (list 1 1 0 0 0))
            (append (list 0) (decode (rest bits)))
        (if (equal (first bits) (list 0 0 0 1 1))
            (append (list 1) (decode (rest bits)))
        (if (equal (first bits) (list 0 0 1 0 1))
            (append (list 2) (decode (rest bits)))
        (if (equal (first bits) (list 0 0 1 1 0))
            (append (list 3) (decode (rest bits)))
        (if (equal (first bits) (list 0 1 0 0 1))
            (append (list 4) (decode (rest bits)))
        (if (equal (first bits) (list 0 1 0 1 0))
            (append (list 5) (decode (rest bits)))
        (if (equal (first bits) (list 0 1 1 0 0))
            (append (list 6) (decode (rest bits)))
        (if (equal (first bits) (list 1 0 0 0 1))
            (append (list 7) (decode (rest bits)))
        (if (equal (first bits) (list 1 0 0 1 0))
            (append (list 8) (decode (rest bits)))
            (append (list 9) (decode (rest bits))))))))))))))

(check-expect (decode '((0 0 0 1 1)(0 0 1 0 1))) '(1 2))
(check-expect (decode 'nil) 'nil)
(check-expect (decode '((1 0 0 1 0) (0 0 1 0 1) (1 1 0 0 0) (0 1 1 0 0) (0 0 1 1 0))) '(8 2 0 6 3))
(check-expect (decode '((0 0 1 1 0))) '(3))
(check-expect (decode '((1 0 1 0 0)(1 0 1 0 0)(1 0 1 0 0)(1 0 1 0 0)(1 0 1 0 0))) '(9 9 9 9 9))

ACL2S !>>(DEFSNAPSHOT TODO-15)

Summary
Form:  ( DEFLABEL TODO-15 ...)
Rules: NIL
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
 TODO-15
ACL2S !>>(DEFDATA BIT (ONEOF 0 1))
 Predicate events...
Form:  ( DEFTHM BITP-TESTTHM ...)
Form:  ( IN-THEORY (DISABLE* ...))
Form:  ( IN-THEORY (ENABLE ...))
Form:  ( TABLE ACL2::RULESET-TABLE ...)
Form:  ( MAKE-EVENT (LET* ...))
 Tau characterization events...
Defdata/Note: BITP relatively complete for Tau.
Form:  ( DEFTHM ACL2::DEF=>BIT ...)
Form:  ( DEFTHM ACL2::BIT=>DEF ...)
 Enumerator events...
Form:  ( DEFUN NTH-BIT-BUILTIN ...)
Form:  ( DEFUN NTH-BIT/ACC-BUILTIN ...)
Form:  ( PROGN (SET-BOGUS-DEFUN-HINTS-OK T) ...)
Form:  ( ENCAPSULATE NIL (LOGIC) ...)
Time:  0.06 seconds (prove: 0.02, print: 0.00, other: 0.03)
 Registering type...
Form:  ( DEFUN NTH-BIT ...)
Form:  ( ENCAPSULATE (((NTH-BIT * ...) ...) ...) ...)
Form:  ( DEFUN NTH-BIT/ACC ...)
Form:  ( ENCAPSULATE (((NTH-BIT/ACC * ...) ...) ...) ...)
Form:  ( DEFATTACH (NTH-BIT N

### TODO-16 (10 points)

It should be obviout, right?, that if you encode a message and then decode the result, you get the original message back. Use `thm` to prove this idea in ACL2.



In [16]:
(defsnapshot todo-16)

(thm (implies (list-digitsp digits)
              (equal (decode(encode digits)) digits)))

ACL2S !>>(DEFSNAPSHOT TODO-16)

Summary
Form:  ( DEFLABEL TODO-16 ...)
Rules: NIL
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
 TODO-16
ACL2S !>>(THM (IMPLIES (LIST-DIGITSP DIGITS)
                       (EQUAL (DECODE (ENCODE DIGITS))
                              DIGITS)))

ACL2 Warning [Invariant-risk] in ACL2::TEST-CHECKPOINT:  Invariant-
risk has been detected for a call of function ACL2::TEST-CHECKPOINT
(as possibly leading to an ill-guarded call of CGEN::UI); see :DOC
invariant-risk.


ACL2 Warning [Invariant-risk] in ACL2::TEST-CHECKPOINT:  Invariant-
risk has been detected for a call of function ACL2::TEST-CHECKPOINT
(as possibly leading to an ill-guarded call of CGEN::UI); see :DOC
invariant-risk.


Name the formula above *1.

Perhaps we can prove *1 by induction.  Three induction schemes are
suggested by this conjecture.  Subsumption reduces that number to two.
These merge into one derived induction scheme.  

We will induct according to a scheme suggested by 

### TODO-17 (10 points)

Now it is time to detect one-bit errors. Write a function called `(detect encoded-msg)` that takes in a list of 5-bit words and returns the (zero-based) index of the first digit that has an invalid encoding. Remember that an invalid encoding is one of the ones that corresponds to a digit. If all the digits are valid, the function should return -1.  For example,

    (detect '((1 0 0 1 0) (0 0 1 0 1) (1 1 0 0 0) (0 1 1 0 0) (0 0 1 1 0))) = -1
    (detect '((1 0 0 1 0) (0 0 1 0 1) (1 0 0 0 0) (0 1 1 0 0) (0 0 1 1 0))) = 2



In [17]:
(defsnapshot todo-17)

(definec detect (encoded-msg :bit-list index :int):int
    (if (endp encoded-msg)
        -1
        (if (equal (dotpr (first encoded-msg) '(1 1 1 1 1)) 2)
            (detect (rest encoded-msg) (+ index 1))
            index)))

(check-expect (detect '((1 0 0 1 0) (0 0 1 0 1) (1 1 0 0 0) (0 1 1 0 0) (0 0 1 1 0)) 0) -1)
(check-expect (detect '((1 0 0 1 0) (0 0 1 0 1) (1 0 0 0 0) (0 1 1 0 0) (0 0 1 1 0)) 0) 2)
(check-expect (detect '((1 0 0 1 0) (0 0 1 0 1) (1 1 0 0 0) (0 0 0 0 0) (0 0 1 1 0)) 0) 3)
(check-expect (detect '((1 0 0 1 0) (0 0 1 0 1) (1 1 0 0 0) (0 1 1 0 0) (0 0 1 1 1)) 0) 4)
(check-expect (detect '((1 1 0 0 0)) 0) -1)

ACL2S !>>(DEFSNAPSHOT TODO-17)

Summary
Form:  ( DEFLABEL TODO-17 ...)
Rules: NIL
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
 TODO-17
ACL2S !>>(DEFINEC DETECT (ENCODED-MSG BIT-LIST INDEX INT)
                  INT
                  (IF (ENDP ENCODED-MSG)
                      -1
                      (IF (EQUAL (DOTPR (FIRST ENCODED-MSG) '(1 1 1 1 1))
                                 2)
                          (DETECT (REST ENCODED-MSG) (+ INDEX 1))
                          INDEX)))

Form:  ( TEST-DEFINITION DETECT ... )
Form:  ( TEST-BODY-CONTRACTS DETECT... ) 
Form:  ( TEST-FUNCTION-CONTRACT DETECT ...) 
Testing: Done 
Elapsed Run Time: 1.50 seconds
Form:  ( ADMIT-DEFINITION DETECT ... )
Time:  0.02 seconds (prove: 0.00, print: 0.00, other: 0.02)
Form:  ( PROVE-FUNCTION-CONTRACT DETECT ... )
Time:  0.10 seconds (prove: 0.04, print: 0.00, other: 0.06)
Form:  ( PROVE-BODY-CONTRACTS DETECT ... )
Time:  0.69 seconds (prove: 0.61, print: 0.00, other: 0.08)
Elapsed Run 

### TODO-18 (10 points)

We will need a version of `change-nth-digit-to` that works on bits. Define the function `(change-nth-bit encoded-msg k n)` that flips the nth bit of the kth word in `encocded-msg`. E.g.,'

    (change-nth-bit '((1 0 0 1 0) (0 0 1 0 1) (1 1 0 0 0) (0 1 1 0 0) (0 0 1 1 0)) 2 1)
                  = '((1 0 0 1 0) (0 0 1 0 1) (1 0 0 0 0) (0 1 1 0 0) (0 0 1 1 0))

Note that since the bits can only be 0 or 1, there is no need to specify what the new value of the bit should be! Also, the result of calling `change-nth-bit` with legal arguments (i.e., `k` and `n` in range) will always be different than the original message. This will simplify the way we need to state some properties later.

In [18]:
(defsnapshot todo-18)

(defdata bits2
    (listof bit))

(defdata bit-list2
    (listof bits2))

(definec flip-nth-bit-aux (encoded-msg :bits2 b :nat):bits2
    (if (< (len encoded-msg) (+ b 1))
        encoded-msg
        (if (or (equal b 0) (endp encoded-msg))
            (if (equal (first encoded-msg) 1)
                (cons 0 (rest encoded-msg))
                (cons 1 (rest encoded-msg)))
            (append (list (first encoded-msg)) (flip-nth-bit-aux (rest encoded-msg) (- b 1))))))

(check-expect (flip-nth-bit-aux '(1 0 0 0 0) 1) '(1 1 0 0 0))
(check-expect (flip-nth-bit-aux '(1 0 0 0 0) 4) '(1 0 0 0 1))
(check-expect (flip-nth-bit-aux '(1 0 0 0 0) 2) '(1 0 1 0 0))
(check-expect (flip-nth-bit-aux '(1 0 0 0 0) 7) '(1 0 0 0 0))
(check-expect (flip-nth-bit-aux '(1 0 0 0 0) 3) '(1 0 0 1 0))

(definec flip-nth-bit (encoded-msg :bit-list2 n :nat b :nat):bit-list2
    (if (< (len encoded-msg) (+ n 1))
        encoded-msg
        (if (or (equal n 0) (endp encoded-msg))
            (append (list (flip-nth-bit-aux (first encoded-msg) b)) (rest encoded-msg))
            (append (list (first encoded-msg)) (flip-nth-bit (rest encoded-msg) (- n 1) b)))))

(check-expect (flip-nth-bit '((1 1 0 0 0)(1 0 0 0 0)) 1 1) '((1 1 0 0 0)(1 1 0 0 0)))
(check-expect (flip-nth-bit '((1 0 0 1 0) (0 0 1 0 1) (1 1 0 0 0) (0 1 1 0 0) (0 0 1 1 0)) 2 1)
              '((1 0 0 1 0) (0 0 1 0 1) (1 0 0 0 0) (0 1 1 0 0) (0 0 1 1 0)))
(check-expect (flip-nth-bit '((1 1 0 0 0)(1 0 0 0 0)) 1 4) '((1 1 0 0 0)(1 0 0 0 1)))
(check-expect (flip-nth-bit '((1 1 0 0 1)) 0 1) '((1 0 0 0 1)))
(check-expect (flip-nth-bit '((1 1 0 0 1)(1 0 0 1 0)) 0 4) '((1 1 0 0 0)(1 0 0 1 0)))

ACL2S !>>(DEFSNAPSHOT TODO-18)

Summary
Form:  ( DEFLABEL TODO-18 ...)
Rules: NIL
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
 TODO-18
ACL2S !>>(DEFDATA BITS2 (LISTOF BIT))
 Predicate events...
Form:  ( DEFUN BITS2P ...)
Form:  ( IN-THEORY (DISABLE* ...))
Form:  ( IN-THEORY (ENABLE ...))
Form:  ( TABLE ACL2::RULESET-TABLE ...)
Form:  ( MAKE-EVENT (LET* ...))
 Listof theory events...
Form:  ( DEFTHM BITS2P-IMPLIES-TLP ...)
Form:  ( DEFTHM BITS2P-SUBTYPE-OF-ATOM-LIST ...)
Form:  ( TABLE ACL2::RULESET-TABLE ...)
Form:  ( MAKE-EVENT (LET* ...))
 Tau characterization events...
Defdata/Note: BITS2P relatively complete for Tau.
Form:  ( DEFTHM DEF=>BITS2 ...)
Form:  ( DEFTHM BITS2=>DEF ...)
 Enumerator events...
Form:  ( DEFUN NTH-BITS2-BUILTIN ...)
Form:  ( DEFUN NTH-BITS2/ACC-BUILTIN ...)
Form:  ( PROGN (SET-BOGUS-DEFUN-HINTS-OK T) ...)
Form:  ( ENCAPSULATE NIL (LOGIC) ...)
Time:  0.18 seconds (prove: 0.11, print: 0.00, other: 0.06)
 Registering type...
Form:  ( DEFUN NTH-BI

### TODO-19 (10 points)

Suppose we have a message $M$, then convert to a list of 5-bit words $A$. We take a copy of that (encoded) message $A$ to another user, and possibly the copy $A'$ has a one-bit error. (You should be thinkinng about some nested calls of functions you already defined in ACL2.)

Define a function called `(correct encoded-msg-copy)` that decodes the encoded msg into a list of digits, but correcting the `encoded-msg` so that it recovers from a one-bit error. Here's how it does it:

1. First call `detect` to find if one of the digits in `encoded-msg-copy`.
2. Also, convert the `encoded-msg-copy` to a list of digits using `decode`. Note that if there is an error, that 
specific digit will be a 0 (because of the way we wrote `decode`.)
3. If no errors are detected, then the answer is the list of digits that `decode` returned.
4. If an error was made, then compute the correct checksum for the current list. That should be the the number that you need to add to sum of the digits to get 0 (mod 10). E.g., if the digits add up to 17, then the correct checksum is 3.
5. The correct answer is the decode list, but changing the incorrect digit to be this checksum.

In [19]:
(defsnapshot todo-19)

(definec sum-msg (msg :list-digits) :nat
    (if (endp msg)
        0
        (+ (first msg) (sum-msg (rest msg)))))

(definec compute-check-sum(msg :list-digits) :digit
    (- 9 (mod (sum-msg msg) 10)))

(definec change-nth-digit-to-2 (ticket :list-digits n :int d :digit):list-digits
    (if (< (len ticket) (+ n 1))
        ticket
        (if (or (equal n 0) (endp ticket))
            (cons d (rest ticket))
            (append (list (first ticket)) (change-nth-digit-to-2 (rest ticket) (- n 1) d)))))

(definec correct-copy (msg :bit-list) :list-digits
    (if (equal (detect msg 0) -1)
        (decode msg)
        (change-nth-digit-to-2 (decode msg) (detect msg 0) (compute-check-sum (decode msg)))))

;sum message
(check-expect (sum-msg '(1 2 3)) 6)
(check-expect (sum-msg '(3)) 3)
(check-expect (sum-msg '(5 4 9)) 18)
(check-expect (sum-msg '(3 8 5 3)) 19)
(check-expect (sum-msg '(0 0 0 0 0 0 0 0 0)) 0)

;compute check sum
(check-expect (compute-check-sum '(5 4 9)) 1)
(check-expect (compute-check-sum '(3 2 9)) 5)
(check-expect (compute-check-sum '(5 9 5)) 0)
(check-expect (compute-check-sum '(1 1 9)) 8)
(check-expect (compute-check-sum '(5 1 9 5 5 5 9)) 0)

;change nth digit 2.0
(check-expect (change-nth-digit-to-2 '(1 2 3 4 5) 2 4) '(1 2 4 4 5))
(check-expect (change-nth-digit-to-2 '(0 9 8) 2 4) '(0 9 4))
(check-expect (change-nth-digit-to-2 '(1 2) 1 3) '(1 3))
(check-expect (change-nth-digit-to-2 '(1 2 3 4 5 6 7 8 9 0) 8 4) '(1 2 3 4 5 6 7 8 4 0))
(check-expect (change-nth-digit-to-2 '(9 4 2 5 6 7 8 3 4 5 6 7 2 1 3) 12 0) '(9 4 2 5 6 7 8 3 4 5 6 7 0 1 3))


;compute-check-sum
(check-expect (correct-copy '((0 1 0 1 0)(0 1 0 0 1)(1 0 0 0 0))) '(5 4 1))
(check-expect (correct-copy '((0 0 1 1 0) (0 0 1 0 1) (1 1 0 1 0))) '(3 2 5))
(check-expect (correct-copy '((1 0 1 0 0)(0 0 0 0 1))) '(9 1))
(check-expect (correct-copy '((0 1 0 1 0)(0 0 0 1 1)(1 0 1 0 0)(0 1 0 1 0)(0 1 0 1 0)(0 1 0 1 0)(1 0 1 0 1))) '(5 1 9 5 5 5 0))
(check-expect (correct-copy '((1 0 1 0 0)(1 0 1 0 0))) '(9 9))


ACL2S !>>(DEFSNAPSHOT TODO-19)

Summary
Form:  ( DEFLABEL TODO-19 ...)
Rules: NIL
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
 TODO-19
ACL2S !>>(DEFINEC SUM-MSG (MSG LIST-DIGITS)
                  NAT
                  (IF (ENDP MSG)
                      0 (+ (FIRST MSG) (SUM-MSG (REST MSG)))))

Form:  ( TEST-DEFINITION SUM-MSG ... )
Form:  ( TEST-BODY-CONTRACTS SUM-MSG... ) 
Form:  ( TEST-FUNCTION-CONTRACT SUM-MSG ...) 
Testing: Done 
Elapsed Run Time: 0.54 seconds
Form:  ( ADMIT-DEFINITION SUM-MSG ... )
Time:  0.01 seconds (prove: 0.00, print: 0.00, other: 0.01)
Form:  ( PROVE-FUNCTION-CONTRACT SUM-MSG ... )
Time:  0.11 seconds (prove: 0.06, print: 0.00, other: 0.05)
Form:  ( PROVE-BODY-CONTRACTS SUM-MSG ... )
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
Elapsed Run Time: 0.18 seconds
Function Name : SUM-MSG 
Termination proven -------- [*] 
Function Contract proven -- [*] 
Body Contracts proven ----- [*]
 T
ACL2S !>>(DEFINEC COMPUTE-CHECK-SUM (MSG LIS

### TODO-20 (10 points)

Now, use `test?` to verify that when you encode message $A$, and $A'$ is either equal to $A$ or equal to the result of changing **one** bit in $A$, then when `(correct A')` is equal to $A$. Compare this with TODO-16 which considered the simpler case when $A'$ was always equal to $A$. We assume here that $A$ is a valid message, i.e., the sum of its digits modulo 10 is equal to 0.

So there you have it. This postnet scheme really can recover the original message, even when a single bit is corrupted in transmission (or an envelope is smudged with dirt in one place)!

In [20]:
(defsnapshot todo-20)

(test? (implies (and (bit-listp message) (natp bitGroup) (natp bit) (< bitGroup (- (len message) 1)) (< bit 5) 
                     (equal (correct-copy message) (decode message)))
                (or (equal (correct-copy (flip-nth-bit message bitGroup bit)) (decode message))
                    (equal (correct-copy message) (decode message)))))

(thm (implies (and (bit-listp message) (natp bitGroup) (natp bit) (< bitGroup (- (len message) 1)) (< bit 5) 
                     (equal (correct-copy message) (decode message)))
                (or (equal (correct-copy (flip-nth-bit message bitGroup bit)) (decode message))
                    (equal (correct-copy message) (decode message)))))

ACL2S !>>(DEFSNAPSHOT TODO-20)

Summary
Form:  ( DEFLABEL TODO-20 ...)
Rules: NIL
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
 TODO-20
ACL2S !>>(TEST?
           (IMPLIES
                (AND (BIT-LISTP MESSAGE)
                     (NATP BITGROUP)
                     (NATP BIT)
                     (< BITGROUP (- (LEN MESSAGE) 1))
                     (< BIT 5)
                     (EQUAL (CORRECT-COPY MESSAGE)
                            (DECODE MESSAGE)))
                (OR (EQUAL (CORRECT-COPY (FLIP-NTH-BIT MESSAGE BITGROUP BIT))
                           (DECODE MESSAGE))
                    (EQUAL (CORRECT-COPY MESSAGE)
                           (DECODE MESSAGE)))))

**Summary of Cgen/testing**
We tested 2000 examples across 2 subgoals, of which 0 (0 unique) satisfied
the hypotheses, and found 0 counterexamples and 0 witnesses.

Test? proved the conjecture under consideration. Therefore, no counterexamples
exist. 
ACL2S !>>(THM
           (IMPLIES
            

## Extra-Credit UPCs (up to 50 points)

Look up how UPC codes are encoded, e.g., in the [Wikipedia entry](https://en.wikipedia.org/wiki/Universal_Product_Code#Check_digit_calculation). Then repeat ther steps todo-6, todo-7, and todo-8, but for UPC codes.

Do you expect counbterexamples in todo-7 and todo-8? Think about our discussions regarding relative prime weights and difference of adjacent weights!

In [21]:
(defsnapshot ec-1)

(definec valid-upc-a (upc-a :list-digits):bool
    (if (equal (len upc-a) 12)
        (equal (mod (dotpr upc-a '(3 1 3 1 3 1 3 1 3 1 3 1)) 10) 0)
        nil))

(check-expect (valid-upc-a '(1 2 2 1 5 0 2 7 8 0 0 6)) t)
(check-expect (valid-upc-a '(1 1 0 0 0 0 0 0 0 0 9 9)) t)
(check-expect (valid-upc-a '(1 2 2 1 5 8 2 7 0 9 1 0)) t)
(check-expect (valid-upc-a '(1 2 3 1 5 0 2 7 8)) nil)
(check-expect (valid-upc-a '(0 3 6 0 0 0 2 9 1 4 5 2)) t)

(test? (implies (and (list-digitsp upc-a) (natp nthDigit) (digitp newDigit) (< nthDigit (len upc-a)) (equal (len upc-a) 12)
                     (not (equal (get-nthDigit upc-a nthDigit) newDigit))
                     (equal (valid-upc-a upc-a) t))
                (equal (valid-upc-a (change-nth-digit-to upc-a nthDigit newDigit)) nil)))

;Since adjacent weights are different, most transposition errors should be detectable
(test? (implies (and (list-digitsp upc-a) (natp nthDigit) (< nthDigit (len upc-a)) (equal (len upc-a) 12)
                     (equal (valid-upc-a upc-a) t))
                (equal (valid-upc-a (transpose-nth-digit upc-a nthDigit)) nil)))

ACL2S !>>(DEFSNAPSHOT EC-1)

Summary
Form:  ( DEFLABEL EC-1 ...)
Rules: NIL
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
 EC-1
ACL2S !>>(DEFINEC VALID-UPC-A (UPC-A LIST-DIGITS)
                  BOOL
                  (IF (EQUAL (LEN UPC-A) 12)
                      (EQUAL (MOD (DOTPR UPC-A '(3 1 3 1 3 1 3 1 3 1 3 1))
                                  10)
                             0)
                      NIL))

Form:  ( TEST-DEFINITION VALID-UPC-A ... )
Form:  ( TEST-BODY-CONTRACTS VALID-UPC-A... ) 
Form:  ( TEST-FUNCTION-CONTRACT VALID-UPC-A ...) 
Testing: Done 
Elapsed Run Time: 0.21 seconds
Form:  ( ADMIT-DEFINITION VALID-UPC-A ... )
Time:  0.01 seconds (prove: 0.00, print: 0.00, other: 0.01)
Form:  ( PROVE-FUNCTION-CONTRACT VALID-UPC-A ... )
Time:  0.09 seconds (prove: 0.03, print: 0.00, other: 0.06)
Form:  ( PROVE-BODY-CONTRACTS VALID-UPC-A ... )
Time:  0.01 seconds (prove: 0.00, print: 0.00, other: 0.01)
Elapsed Run Time: 0.17 seconds
Function Name : VALID-UPC-

# Test?
## Airline Tickets
#### Change Digit
 `(test? (implies (and (list-digitsp ticket) (natp nthDigit) (digitp newDigit) (< nthDigit (len ticket)) (equal (len ticket) 15)
                     (not (equal (get-nthDigit ticket nthDigit) newDigit))
                     (equal (valid-ticket ticket) t))
                (equal (valid-ticket (change-nth-digit-to ticket nthDigit newDigit)) nil)))`

Summary: We tested 3000 examples across 3 subgoals, of which 0 (0 unique) satisfied the hypotheses, and found 0 counterexamples and 0 witnesses.
#### Transpose Digit
`(test? (implies (and (list-digitsp ticket) (natp nthDigit) (< nthDigit (len ticket)) (equal (len ticket) 15)
                     (equal (valid-ticket ticket) t))
                (equal (valid-ticket (transpose-nth-digit ticket nthDigit)) nil)))`
                
Summary: We tested 3000 examples across 3 subgoals, of which 0 (0 unique) satisfied the hypotheses, and found 0 counterexamples and 0 witnesses.
***
## Bank Routing Numbers
#### Change Digit
`(test? (implies (and (list-digitsp routNum) (natp nthDigit) (digitp newDigit) (< nthDigit (len routNum)) (equal (len routNum) 9)
                     (not (equal (get-nthDigit routNum nthDigit) newDigit))
                     (equal (valid-routing-number routNum) t))
                (equal (valid-routing-number (change-nth-digit-to routNum nthDigit newDigit)) nil)))`

Summary: We tested 3000 examples across 3 subgoals, of which 0 (0 unique) satisfied the hypotheses, and found 0 counterexamples and 0 witnesses.
#### Transpose Digit
`(test? (implies (and (list-digitsp routNum) (natp nthDigit) (< nthDigit (len routNum)) (equal (len routNum) 9)
                     (not (equal (get-nthDigit routNum nthDigit) newDigit))
                     (equal (valid-routing-number routNum) t))
                (equal (valid-routing-number (transpose-nth-digit routNum nthDigit)) nil)))`
                
Summary: We tested 3000 examples across 3 subgoals, of which 0 (0 unique) satisfied the hypotheses, and found 0 counterexamples and 0 witnesses.
***
## Credit Card Numbers
#### Change Digit
`(test? (implies (and (list-digitsp ccard) (natp nthDigit) (digitp newDigit) (< nthDigit (len ccard)) (equal (len ccard) 16)
                     (not (equal (get-nthDigit ccard nthDigit) newDigit))
                     (equal (valid-ccard-number ccard) t))
                (equal (valid-ccard-number (change-nth-digit-to ccard nthDigit newDigit)) nil)))`
Summary: We tested 6000 examples across 6 subgoals, of which 0 (0 unique) satisfied the hypotheses, and found 0 counterexamples and 0 witnesses.
#### Transpose Digit
`(test? (implies (and (list-digitsp ccard) (natp nthDigit) (< nthDigit (len ccard)) (equal (len ccard) 16)
                     (equal (valid-ccard-number ccard) t))
                (equal (valid-ccard-number (transpose-nth-digit ccard nthDigit)) nil)))`

Summary: We tested 6000 examples across 6 subgoals, of which 0 (0 unique) satisfied the hypotheses, and found 0 counterexamples and 0 witnesses.
***
## ISBN
#### Change Digit
`(test? (implies (and (isbnp isbn) (natp nthDigit) (x-or-digitp newDigit) (< nthDigit (len isbn)) (equal (len isbn) 10)
                     (not (equal (get-nthDigit isbn nthDigit) newDigit))
                     (equal (valid-isbn isbn) t))
                (equal (valid-isbn (change-nth-digit-to isbn nthDigit newDigit)) nil)))`
                

Summary: We tested 4000 examples across 4 subgoals, of which 0 (0 unique) satisfied the hypotheses, and found 0 counterexamples and 0 witnesses.
#### Tranpose Digit
`(test? (implies (and (isbnp isbn) (natp nthDigit) (x-or-digitp newDigit) (< nthDigit (len isbn)) (equal (len isbn) 10)
                     (equal (valid-isbn isbn) t))
                (equal (valid-isbn (transpose-nth-digit isbn nthDigit)) nil)))`
                
Summary: We tested 4000 examples across 4 subgoals, of which 0 (0 unique) satisfied the hypotheses, and found 0 counterexamples and 0 witnesses.
***
## Messages
#### Detect and Correct Change
`(test? (implies (and (bit-listp message) (natp bitGroup) (natp bit) (< bitGroup (- (len message) 1)) (< bit 5) 
                     (equal (correct-copy message) (decode message)))
                (or (equal (correct-copy (flip-nth-bit message bitGroup bit)) (decode message))
                    (equal (correct-copy message) (decode message)))))`
                    
Summary: We tested 2000 examples across 2 subgoals, of which 0 (0 unique) satisfied the hypotheses, and found 0 counterexamples and 0 witnesses.
****
## UPC-A
#### Change Digit
`(test? (implies (and (list-digitsp upc-a) (natp nthDigit) (digitp newDigit) (< nthDigit (len upc-a)) (equal (len upc-a) 12)
                     (not (equal (get-nthDigit upc-a nthDigit) newDigit))
                     (equal (valid-upc-a upc-a) t))
                (equal (valid-upc-a (change-nth-digit-to upc-a nthDigit newDigit)) nil)))`
                
Summary: We tested 3000 examples across 3 subgoals, of which 0 (0 unique) satisfied the hypotheses, and found 0 counterexamples and 0 witnesses.
#### Transpose Digit
`(test? (implies (and (list-digitsp upc-a) (natp nthDigit) (< nthDigit (len upc-a)) (equal (len upc-a) 12)
                     (equal (valid-upc-a upc-a) t))
                (equal (valid-upc-a (transpose-nth-digit upc-a nthDigit)) nil)))`

Summary: We tested 3000 examples across 3 subgoals, of which 0 (0 unique) satisfied the hypotheses, and found 0 counterexamples and 0 witnesses.